# Getting started with the CMSIS-DSP library


To get started, create a new STM32 project for the STM32 F446RE Nucleo board, and initialize all peripherals with their default values. Build the code by clicking **Project->Generate Code**.

### a) 
#### Adding DSP libraries to the stm32 project

The process of adding the DSP libraries from ARM is rather cumbersome as described on [this page](https://community.st.com/t5/stm32-mcus/how-to-integrate-cmsis-dsp-libraries-on-a-stm32-project/ta-p/666790), we will be taking a shortcut:

* Attached to this assignment is a file `CMSIS_DSP.zip` which contains the modules we need to implement our frequency detection algorithm. Unpack folder named `CMSIS_DSP` into the `Drivers` folder within your STM32 project. Then right-click on your project and select `Refresh`. The folder structure for your project should now look something like [this](Figures/Folder_structure.png).
* Right click on your project and select `Properties`. Navigate to `C/C++ Build -> Settings -> MCU GCC Compiler -> Include paths`. Add the lines `../Drivers/CMSIS_DSP/Include` and `../Drivers/CMSIS_DSP/PrivateInclude` to the include paths. The final settings should look like [this](Figures/IncludePaths.png).
* Verify that the library has been successfully included by including the `arm_math.h` file to the private includes section of `main.c`, and running a build of the project:
```C
/* Private includes ----------------------------------------------------------*/
/* USER CODE BEGIN Includes */
#include "arm_math.h"
/* USER CODE END Includes */
```

## About the CMSIS-DSP library

The CMSIS-DSP library contains a large number of new functions, structs and data types. Generally the new data types such as `flaot32_t` are simply an alias standard data types such as `float`, but to maintain good pratice we will use the `float32_t` data type. That way, the variable declarations will "match" the functions with suffic `_f32` in their function name.

The functions themselves are created to process entire arrays of data rather than individual values, similar in spirit to the `numpy` library. However, since this is  `C` and not `Python`, the functions can't return entire arrays as output and instead rely on [*pointers*](https://www.w3schools.com/c/c_pointers.php) to where they should write the calculated output. Below is an example of using a simple copy-function for an array of 32-bit floating point numbers:

> #### Code Example:
>```C
>#define ARR_SIZE 4 
>float32_t input_array[ARR_SIZE] = {1.0, 2.0, 3.0, 4.0};
>float32_t output_array[ARR_SIZE];
>
>arm_copy_f32(input_array, output_array, ARR_SIZE);
>
>```
>
>#### Explanation:
> The function `arm_copy_f32` has return type `void`, meaning it does not return a value. Instead, it recieves a pointer to the array `output_array`, whose values will be changed by the execution of the function. In other words, once the function call is executed, the content of  `input_array` is copied to `output_array`. Since a pointer is merely a reference to "where the array starts" with no added information about e.g. length, we also need to pass an argument `ARR_SIZE` so that the function can know how long the arrays are.

### Processing complex data

Many of the functions in the CMSIS-DSP library use complex numbers as input/output. Complex data is stored in a normal array with length $2\ \times$ the number of complex values. Real and complex parts are then stored alternately as follows:

#### Array representing complex data $z[n] = x[n] + j\cdot y[n]$:

|index: | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|:--|--:|--:|--:|--:|--:|--:|--:|--:|
|**data:** | $$x[0]$$| $$y[0]$$| $$x[1]$$| $$y[1]$$| $$x[2]$$| $$y[2]$$| $$x[3]$$| $$y[3]$$|


### b) 
#### Performing simple calculations with complex data

A useful function for our signal processing algorithm is the function `arm_cmplx_mag_f32`, which takes three arguments as input data.
*   `pSrc`        points to input vector with complex data. Expects `float32_t` array with length $2\times$`numSamples`.
*   `pDst`        points to output vector with real data. Expects `float32_t` array with length `numSamples`.
*   `numSamples`  number of samples in each vector

Use the function `arm_cmplx_mag_f32` to calculate the magnitude of a complex value $z = 4 - 3j$ on the microcontroller. You can check the that the calculation was successful by entering debug mode, and checking the value of your output vector after calling `arm_cmplx_mag_f32`. Paste the code you used to the cell below.


```C

//ANSWER QUESTION B) HERE:


```


### Printing results to serial monitor

The goal of this lab is to program the STM32 as a frequency detection system, which transmits a constant stream of updated frequency values in $\text{Hz}$. A logical first step is to program the STM32 to transmit messages to your PC using USART.

By default, USART2 is configured for serial communication with a baud rate of $115\ 200\text{ bits/s}$ (you can adjust this to a higher setting if you wish, although it's not strictly necessary for our application). To transmit messages to the PC we can use the function `HAL_UART_Transmit`, which is as close as it gets to Arduino C's `Serial.print()`. Below is an example of usage:

```C
char message[] = "Hello World!";
HAL_UART_Transmit(&huart2, (uint8_t*) &message, sizeof(message), 10); 
```

Explanation of arguments:
* `&huart2`: Reference to UART "object"
* `message`: Character array with message to be transmitted.
* `(uint8_t*) &message`: Number of bytes in message
* `10`: Timeout. Milliseconds before exiting transmit function.


## c)
Program the STM32 to transmit the result from our complex magnitude calculation in problem **b)**. You can view the transmitted data using e.g. a serial monitor such as the one included in the Arduino IDE.

*Hint: the function `sprintf` from `stdio.h` may come in handy.*